In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Model, Sequential
from keras.layers import Dense, Embedding, SimpleRNN, LSTM, GRU, Bidirectional,InputLayer,TimeDistributed
from sklearn.model_selection import train_test_split

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer, SnowballStemmer


from nltk.corpus import brown, treebank ## it is a huge collection of textual data

In [2]:
treebank.sents()

[['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'], ['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.'], ...]

In [3]:
len(brown.sents()) 
## I"ll take the brown and from that i'll take the sents tokinzation
## also i'll check what is len of the brown
## in this i'am having the 57340 sentences

57340

In [4]:
brown.sents()[0]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of',
 "Atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'no',
 'evidence',
 "''",
 'that',
 'any',
 'irregularities',
 'took',
 'place',
 '.']

In [5]:
" ".join(brown.sents()[0])  ## here i have joined all the words from sentence1 and joined them into a sentence as shown below

"The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place ."

In [6]:
" ".join(brown.sents()[1]) ## here in this we need not to perfrom any tokenization because it only give the tokens

"The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted ."

In [7]:
treebank.sents()

[['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'], ['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.'], ...]

In [8]:
len(treebank.sents())

3914

In [9]:
## in all case we make data set download and run it 
## but we can generate the data set by using the nltk .corpus as shown below


In [10]:
 brown.tagged_sents(tagset="universal")  
## by using this we can generate both the x and y as shown below
## in the same it is done for the tree bank also and the other data which are also present in the nltk

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [11]:
 brown.tagged_sents(tagset="universal")[0] 
## the below is the x and y for the 1st sentence as shown below

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

In [12]:
treebank.tagged_sents(tagset="universal")

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]

In [13]:
treebank.tagged_sents(tagset="universal")[0]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET'),
 ('board', 'NOUN'),
 ('as', 'ADP'),
 ('a', 'DET'),
 ('nonexecutive', 'ADJ'),
 ('director', 'NOUN'),
 ('Nov.', 'NOUN'),
 ('29', 'NUM'),
 ('.', '.')]

In [14]:
d1 = treebank.tagged_sents(tagset="universal")
d1

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]

In [15]:
d2 = brown.tagged_sents(tagset="universal")
d2

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [16]:
## by using the addition operator i will join both d1 and d2 and make the data set
## in this data is present in the form of tuples in the list's
## eg:- [ (sent1 ),(sent2),(sent3),....(nsent)] .. in this way the raw data is present 
## eg:- [[(w1,pos) (w2,pos) (w3,pos)],[(w1,pos) (w2,pos) (w3,pos)]] in this way the raw data is present

dataset = d1+d2
dataset

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]

In [17]:
len(dataset)

61254

In [18]:
57340+3914


61254

In [19]:
## here in this i have maked the dataset from the raw data using the for loop

x=[]
y=[]

for l in dataset:
    x1=[]
    y1=[]
    for t in l:
        x1.append(t[0])
        y1.append(t[1])

    x.append(x1)
    y.append(y1)

print(len(x))
print(len(y))

61254
61254


In [20]:
x

[['Pierre',
  'Vinken',
  ',',
  '61',
  'years',
  'old',
  ',',
  'will',
  'join',
  'the',
  'board',
  'as',
  'a',
  'nonexecutive',
  'director',
  'Nov.',
  '29',
  '.'],
 ['Mr.',
  'Vinken',
  'is',
  'chairman',
  'of',
  'Elsevier',
  'N.V.',
  ',',
  'the',
  'Dutch',
  'publishing',
  'group',
  '.'],
 ['Rudolph',
  'Agnew',
  ',',
  '55',
  'years',
  'old',
  'and',
  'former',
  'chairman',
  'of',
  'Consolidated',
  'Gold',
  'Fields',
  'PLC',
  ',',
  'was',
  'named',
  '*-1',
  'a',
  'nonexecutive',
  'director',
  'of',
  'this',
  'British',
  'industrial',
  'conglomerate',
  '.'],
 ['A',
  'form',
  'of',
  'asbestos',
  'once',
  'used',
  '*',
  '*',
  'to',
  'make',
  'Kent',
  'cigarette',
  'filters',
  'has',
  'caused',
  'a',
  'high',
  'percentage',
  'of',
  'cancer',
  'deaths',
  'among',
  'a',
  'group',
  'of',
  'workers',
  'exposed',
  '*',
  'to',
  'it',
  'more',
  'than',
  '30',
  'years',
  'ago',
  ',',
  'researchers',
  'reported'

In [21]:
y

[['NOUN',
  'NOUN',
  '.',
  'NUM',
  'NOUN',
  'ADJ',
  '.',
  'VERB',
  'VERB',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'NOUN',
  'NUM',
  '.'],
 ['NOUN',
  'NOUN',
  'VERB',
  'NOUN',
  'ADP',
  'NOUN',
  'NOUN',
  '.',
  'DET',
  'NOUN',
  'VERB',
  'NOUN',
  '.'],
 ['NOUN',
  'NOUN',
  '.',
  'NUM',
  'NOUN',
  'ADJ',
  'CONJ',
  'ADJ',
  'NOUN',
  'ADP',
  'NOUN',
  'NOUN',
  'NOUN',
  'NOUN',
  '.',
  'VERB',
  'VERB',
  'X',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'ADJ',
  'NOUN',
  '.'],
 ['DET',
  'NOUN',
  'ADP',
  'NOUN',
  'ADV',
  'VERB',
  'X',
  'X',
  'PRT',
  'VERB',
  'NOUN',
  'NOUN',
  'NOUN',
  'VERB',
  'VERB',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'NOUN',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  'VERB',
  'X',
  'PRT',
  'PRON',
  'ADV',
  'ADP',
  'NUM',
  'NOUN',
  'ADP',
  '.',
  'NOUN',
  'VERB',
  'X',
  'X',
  '.'],
 ['DET',
  'NOUN',
  'NOUN',
  '.',
  'NOUN',
  '.',
  'VERB',
  'ADV',
  'ADJ',
  'ADP'

In [22]:
len(x[0]) == len(y[0])  ## to whether the input is equal output or not

True

In [23]:
## we need to check for all the input wheather the input is equal to output or not 
## by using this we can check for the entire data

c = 0
for i in range(len(x)):
    if len(x[i]) == len(y[i]):
        pass
    else:
        c+=1

c

0

In [24]:
## finding out the volcabulary from the data or unique words from the data

voc = []
for q in x:
    for word in q:
        if word.lower() not in voc:
            voc.append(word.lower())

In [25]:
len(voc)

53232

In [26]:
## finding out the tags in the volcabulary from the data or unique words from the data

voca = []
for q in y:
    for word in q:
        if word.lower() not in voca:
            voca.append(word.lower())

In [27]:
len(voca)

12

In [28]:
## after dividing the raw data into x and y then do the tokenization to convert them into the vector form 

tk_x = Tokenizer()
tk_x.fit_on_texts(x)

tk_y = Tokenizer()
tk_y.fit_on_texts(y)

In [29]:
len(tk_x.word_index)

53232

In [30]:
len(tk_y.word_index)

12

In [31]:
tk_y.word_index

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

#### converting the text into vector using the texts to sequences

In [32]:
x_d = tk_x.texts_to_sequences(x)  ### here we are converting the words into vector using the texts to sequence
x_d

[[5594,
  21728,
  2,
  9068,
  105,
  158,
  2,
  53,
  1748,
  1,
  344,
  18,
  7,
  13148,
  880,
  2681,
  4000,
  3],
 [96, 21728, 10, 957, 4, 29496, 17598, 2, 1, 5595, 3617, 222, 3],
 [11765,
  29497,
  2,
  5369,
  105,
  158,
  5,
  759,
  957,
  4,
  9782,
  1934,
  1425,
  7085,
  2,
  12,
  1123,
  101,
  7,
  13148,
  880,
  4,
  25,
  914,
  678,
  17599,
  3],
 [7,
  276,
  4,
  7477,
  199,
  169,
  113,
  113,
  6,
  124,
  4796,
  3903,
  7964,
  48,
  1170,
  7,
  190,
  2162,
  4,
  3443,
  6734,
  250,
  7,
  222,
  4,
  1110,
  3291,
  113,
  6,
  16,
  57,
  67,
  721,
  105,
  380,
  2,
  5596,
  732,
  102,
  135,
  3],
 [1,
  7477,
  4001,
  2,
  13149,
  2,
  10,
  7086,
  29498,
  199,
  16,
  5857,
  1,
  4952,
  2,
  17,
  92,
  1550,
  13150,
  6,
  16,
  4639,
  6430,
  9,
  135,
  355,
  64,
  3205,
  252,
  2,
  5596,
  52,
  102,
  311,
  3],
 [13151,
  1012,
  2,
  1,
  835,
  4,
  68,
  13152,
  29499,
  922,
  9,
  311,
  613,
  4796,
  5597,
  2,


In [33]:
 ### here we are converting the words into vector using the texts to sequence

y_d = tk_y.texts_to_sequences(y)
y_d

[[1, 1, 3, 11, 1, 6, 3, 2, 2, 5, 1, 4, 5, 6, 1, 1, 11, 3],
 [1, 1, 2, 1, 4, 1, 1, 3, 5, 1, 2, 1, 3],
 [1,
  1,
  3,
  11,
  1,
  6,
  9,
  6,
  1,
  4,
  1,
  1,
  1,
  1,
  3,
  2,
  2,
  12,
  5,
  6,
  1,
  4,
  5,
  6,
  6,
  1,
  3],
 [5,
  1,
  4,
  1,
  7,
  2,
  12,
  12,
  10,
  2,
  1,
  1,
  1,
  2,
  2,
  5,
  6,
  1,
  4,
  1,
  1,
  4,
  5,
  1,
  4,
  1,
  2,
  12,
  10,
  8,
  7,
  4,
  11,
  1,
  4,
  3,
  1,
  2,
  12,
  12,
  3],
 [5,
  1,
  1,
  3,
  1,
  3,
  2,
  7,
  6,
  4,
  8,
  2,
  5,
  1,
  3,
  4,
  7,
  6,
  1,
  10,
  8,
  2,
  1,
  5,
  12,
  2,
  10,
  1,
  6,
  3,
  1,
  2,
  12,
  12,
  3],
 [1,
  1,
  3,
  5,
  1,
  4,
  6,
  6,
  1,
  1,
  5,
  12,
  2,
  1,
  1,
  3,
  2,
  2,
  1,
  4,
  8,
  1,
  1,
  1,
  4,
  11,
  3],
 [4,
  6,
  1,
  2,
  2,
  12,
  7,
  4,
  5,
  1,
  4,
  3,
  5,
  6,
  1,
  2,
  4,
  1,
  10,
  1,
  1,
  1,
  4,
  1,
  3,
  5,
  1,
  6,
  12,
  10,
  2,
  6,
  1,
  10,
  5,
  1,
  3],
 [5, 1, 1, 2, 3, 3, 5, 2, 5, 6, 1, 3]

In [34]:
print(len(x_d))
print(len(y_d))

61254
61254


In [48]:
## the reason why we use the padding means to don't loss the information from the data

final_x_d = pad_sequences(x_d,padding='post')  
final_y_d = pad_sequences(y_d,padding='post')

In [36]:
final_x_d.shape

(61254, 271)

In [37]:
final_y_d.shape

(61254, 271)

In [38]:
final_y_d 


array([[1, 1, 3, ..., 0, 0, 0],
       [1, 1, 2, ..., 0, 0, 0],
       [1, 1, 3, ..., 0, 0, 0],
       ...,
       [5, 1, 4, ..., 0, 0, 0],
       [8, 2, 5, ..., 0, 0, 0],
       [4, 5, 8, ..., 0, 0, 0]])

In [39]:
from keras.utils import to_categorical

In [58]:
final_y = to_categorical(final_y_d)
final_y

# in the final_y_d it is not better keep the integer because after conveting them into the numbes
## we get an additional values or the new value is added that is Zero[0]
## so to prevent that one it is better to use the onehotencoding so that it will include the zero[0] 
## it total makes the final_y_d to the 13 dim or class

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [52]:
len(voc)

53232

In [53]:
model=Sequential()
model.add(InputLayer(input_shape=(271,)))
model.add(Embedding(53232+1,5)) ## normal embedding where in the enbrdding we need to use len of the vocabulary
model.add(SimpleRNN(units=50, return_sequences=True)) ## here return sequences is TRUE because we are using the it for multiple rnn's
model.add(TimeDistributed(Dense(units=13,activation='softmax'))) 
## here we used the timedistributed because in the old version without using the timedistribution we can't get teh output
## but now without using the timedistribution also we are getting the output

In [54]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 271, 5)              │         266,165 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 271, 50)             │           2,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 271, 13)             │             663 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 269,628 (1.03 MB)

 Trainable params: 269,628 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
model.compile(optimizer="adam",loss = "categorical_crossentropy",metrics=["accuracy"])

In [56]:
final_y = np.array(final_y)

In [59]:
model.fit(final_x_d, final_y, epochs=10, batch_size=120, validation_split=0.2)

Epoch 1/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 374s 907ms/step - accuracy: 0.9439 - loss: 0.1865 - val_accuracy: 0.9724 - val_loss: 0.0859
Epoch 2/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 350s 856ms/step - accuracy: 0.9698 - loss: 0.1000 - val_accuracy: 0.9866 - val_loss: 0.0470
Epoch 3/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 466s 1s/step - accuracy: 0.9857 - loss: 0.0533 - val_accuracy: 0.9935 - val_loss: 0.0250
Epoch 4/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 353s 863ms/step - accuracy: 0.9921 - loss: 0.0302 - val_accuracy: 0.9955 - val_loss: 0.0170
Epoch 5/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 332s 808ms/step - accuracy: 0.9947 - loss: 0.0196 - val_accuracy: 0.9962 - val_loss: 0.0135
Epoch 6/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 332s 809ms/step - accuracy: 0.9959 - loss: 0.0150 - val_accuracy: 0.9965 - val_loss: 0.0117
Epoch 7/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 332s 812ms/step - accuracy: 0.9965 - loss: 0.0119 - val_accuracy: 0.9966 - val_loss: 0.0106
Epoch 8/10
409/409 ━━━━━━━━━━━━━━━━━━━━ 324s 791ms/step - accuracy: 0.9968 - lo

In [65]:
te = ["Dhoni loves playing cricket and he is from ranchi"]   ## the input we are giving to the model

In [66]:
tk_x.texts_to_sequences(te)  ## converting the input to the numerical

[[5514, 1116, 20351, 5, 13, 10, 29]]

In [67]:
tk_x.sequences_to_texts(tk_x.texts_to_sequences(te)) ## it gives the word which are present in our data vocabulary

## if the word is not present in the vocabulary then in that place it returns the oov(out of vacobulary)
## but here I have not used the oov nut is should here 

['loves playing cricket and he is from']

In [71]:
test = pad_sequences(tk_x.texts_to_sequences(te), maxlen=271,padding="post")
test

## it returns the number where our word is present and returns zero where our word is not present

array([[ 5514,  1116, 20351,     5,    13,    10,    29,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [72]:
model.predict(test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


array([[[3.9865449e-03, 1.4410803e-02, 9.6894115e-01, ...,
         1.0316310e-03, 3.6891268e-03, 1.1184262e-03],
        [8.3808607e-04, 9.8867100e-03, 9.8773056e-01, ...,
         1.8054826e-04, 3.9452154e-04, 2.7390916e-04],
        [2.8891631e-03, 8.4518164e-01, 1.3751964e-01, ...,
         6.7818526e-04, 9.4574137e-04, 2.5402086e-03],
        ...,
        [9.9997830e-01, 6.4624269e-06, 4.8220236e-06, ...,
         1.8825381e-06, 5.9331956e-07, 3.7851217e-07],
        [9.9997830e-01, 6.4624269e-06, 4.8220236e-06, ...,
         1.8825381e-06, 5.9331956e-07, 3.7851217e-07],
        [9.9997830e-01, 6.4624269e-06, 4.8220236e-06, ...,
         1.8825381e-06, 5.9331956e-07, 3.7851217e-07]]], dtype=float32)

In [74]:
model.predict(test).shape 
## here we are getting it in the 271 but we want only the word which are present in the data

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


(1, 271, 13)

In [76]:
[np.argmax(model.predict(test)[0],axis=1)!=0] 
## here we are getting true where the value is present 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


[array([ True,  True,  True,  True,  True,  True,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
      

In [79]:
## here by using the masking i got the values of the word which are present 
## justed applied the masking function on the code and got the output

np.argmax(model.predict(test)[0],axis=1)[np.argmax(model.predict(test)[0],axis=1)!=0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([2, 2, 1, 9, 8, 2, 4], dtype=int64)

In [82]:
tk_y.sequences_to_texts([[2, 2, 1, 9, 8, 2, 4]])

['verb verb noun conj pron verb adp']